<a href="https://colab.research.google.com/github/mikeneko-cat/kimagure/blob/main/4_in_a_row_with_gravity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

重力付き四目並べを作る\
条件\
・6行7列の盤面で行う\
・プレイヤーは手を自分で選択することができる\
・コンピューターは選択可能なマスからランダムで選択する\
・選択した場所がおける場所化の判定\
・勝敗の判定\

In [ ]:
import random


#表示盤面　マス目の位置と1シフトするビット数を対応づけた
for_disp = [
    '41','40','39','38','37','36','35',
    '34','33','32','31','30','29','28',
    '27','26','25','24','23','22','21',
    '20','19','18','17','16','15','14',
    '13','12','11','10','9','8','7',
    '6','5','4','3','2','1','0'
]

#盤面表示
def disp_board():
  for i, v in enumerate(for_disp):
    if v == 'o':
      print("\033[48;2;250;70;70m\033[37m",end='') #oの時(\033[48;2;r;g;bm)で背景を赤に変更(\033[37m)で文字を白に変更
    elif v == 'x':
      print("\033[48;2;70;70;225m\033[37m",end='') #xの時(\033[48;2;r;g;bm)で背景を青に変更(\033[37m)で文字を白に変更
    if i in [6,13,20,27,34,41]:
      print('{:>3}\033[0m'.format(v))
    else:
      print('{:>3}\033[0m'.format(v),end=' ')

# 4つ並んだか判定 (^~^)
def check(player):
    #横方向の判定 (ε_ε)
    for i in range(0,36,7): #各行の一番左の数字を指定
      for j in range(i,i+4): #右から比べていくときにそろう可能性のある4番目の数字まで回す(一番下の行だと3番)
        goal = 0b000000000000000000000000000000000000000000 #goalの初期化
        goal = goal | (0b1111 << j) #指定した場所から4つ1を並べる
        if player & goal ==goal:
            return True

    #縦方向の判定 (-\-)
    for i in range(0,21): #上に4つスペースのある20番まで順番に回す
      goal = 0b000000000000000000000000000000000000000000
      for j in range(i,i+22,7): #1つ上のマスの番号になるように7おきに読み取る
        goal = goal | (0b1 << j) #指定した場所の置換
      if player & goal ==goal:
            return True

    #右下から左上にかけての判定 (-_-)
    for i in range(0,4): #可能性のある4列目まで回す
      for j in range(0,3): #可能性のある3行目まで回す
        goal = 0b000000000000000000000000000000000000000000
        for k in range(i+(7*j),i+(7*j)+(3*8)+1,8):#次に選ぶマスは8個後のマスなので8マス置きに読み取る
          goal = goal | (0b1 << k) #指定した場所の置換
        if player & goal ==goal:
            return True

    #左下から右上にかけての判定 (~-~)
    for i in range(3,7):  #可能性のある4列目まで回す
      for j in range(0,3): #可能性のある3行目まで回す
        goal = 0b000000000000000000000000000000000000000000
        for k in range(i+(7*j),i+(7*j)+(3*6)+1,6):#次に選ぶマスは6個後のマスなので8マス置きに読み取る
          goal = goal | (0b1 << k)#指定した場所の置換
        if player & goal ==goal:
            return True

    return False #すべてに当てはまらないとき (>.<)

# 交互に置く(以下再帰呼び出し)
def play(p1, p2, turn):

   # 1. 盤面表示。
    disp_board()
    if check(p2): # 4つ並んで（勝敗が確定して）いたら終了
        # どちらの勝ちか表示するためのif-else文を追加
        if turn:
            print(f'{gote}の勝ち')
        else:
            print(f'{sente}の勝ち')
        return


    board = p1 | p2
    if board == 0b111111111111111111111111111111111111111111: #全部埋まった引き分け
        print('引き分け') # 明記
        return


    # 置ける場所を探す
    w = [i for i in range(42) if ((board & (1 << i)) == 0) ] #値が0の場所
    r=[] #新しいリストの作成
    for i in w:
      if i <=6: #wに含まれる数のうち6以下の数はすべて追加
        r.append(i)
      elif (i >6) & (i-7 not in w): #6以上の数に対しては、一つ下のマスがリストwに入っていないとき(すでにどちらかが石を置いているとき)rに追加
        r.append(i)
      else:
        continue
    # ランダムに1つ選ぶ
    j = random.choice(r) # rの中からランダムに選択した位置を手とする。
    if turn:
        if sente == 'player':
          print(f'{sente}')
          while True:
            print(f'選択可能なマス : {r}')
            j = int(input('置く位置を指定して下さい'))
            if j in r: #選択したマスがリストrに入っているかの確認
              break
            print('その場所にはおけません')
        else:
          print(f'{sente}は{j}に置きました')
        for_disp[(len(for_disp)-1) - j] = 'o'
    else:
        if gote == 'player':
          print(f'{gote}')
          while True:
              print(f'選択可能なマス : {r}')
              j = int(input('置く位置を指定して下さい'))
              if j in r: #選択したマスがリストrに入っているかの確認
                break
              print('その場所にはおけません')
        else:
          print(f'{gote}は{j}に置きました')
        for_disp[(len(for_disp)-1) - j] = 'x'
    play(p2, p1 | (1 << j), not turn)

#先手後手の決定
while True:
  junban=input('先手(1),後手(-1)を選択してください:')
  if junban == '1':
    sente = 'player'
    gote =  'computer'
    break
  elif junban == '-1':
    sente = 'computer'
    gote =  'player'
    break

print('ゲームを始めます')
print('先手'+'\033[48;2;250;70;70m\033[37m' + 'o'+'\033[0m'+f':{sente} 対 後手'+'\033[48;2;70;70;225m\033[37m' +'x' +'\033[0m:' +f':{gote}')
play(0, 0, True)